In [36]:
import os
import sys
import json
from datetime import datetime
import subprocess
import pandas as pd
import plotly.graph_objects as go
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)
import Birdeye.Basics.Master_Functions as Master_Functions
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Generating Data

In [37]:
# Specify the path to the Python file you want to run
python_file_path = 'Dataset_Generator_New_Tokens.py'

# Run the Python file
subprocess.run(['python', python_file_path])

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}

--------------------------------

Running on: solana

--------------------------------

NEW TOKEN FILTERS:

Min Liquidity: 20K
Max Liquidity: 1.00Mil
Min Market Cap: 200K
Max Market Cap: 5.00Mil
Filtering New Tokens for: 0 days, 23 hours, 0 minutes back
Token Launch Liquidity Filter: 10K

--------------------------------
 
 
--------------------------------

https://dexscreener.com/solana/6dcUiKM18eEjezLRQCoPGZL6E18YiV3i15wDtv4kPumP - Market Cap: 4.74Mil
https://dexscreener.com/solana/FiVdFLykumoEL328DrxgGpu9oB55Nohw7uq6Kzi7pump - Market Cap: 3.77Mil
https://dexscreener.com/solana/7R4JxJm2tQM6LoGD42MrpExxMTUnbRivs7ZPpCWmpump - Market Cap: 3.53Mil
https://dexscreener.com/solana/4wNNmyckLNZ8mrFr1DdckWfPWuYJsFaEwZxF2brLpump - Market Cap: 3.25Mil
https://dexscreener.com/solana/97TW2UWw2pBPEQGY8BZk9vGwPXMJmpnJ3RThFWLunD6j - Market Cap: 3.22Mil
https://dexscreener.com/solana/EkgmX

CompletedProcess(args=['python', 'Dataset_Generator_New_Tokens.py'], returncode=0)

# Import Data

In [38]:
# Get the most recent folder
base_path = 'Data/New_Token_Data'
current_date = datetime.now().strftime('%Y_%m_%d')
date_folder = os.path.join(base_path, current_date)
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
ohlcv_datetime_folder = Master_Functions.get_most_recent_folder(ohlcv_folder)

# Print the name of the folder being used for import
print(f"Importing OHLCV data from folder: {ohlcv_datetime_folder}")

# Import the OHLCV data
imported_ohlcv_data = Master_Functions.import_ohlcv_data(ohlcv_datetime_folder)

print(
    """
    
Enhanced Fibonacci Analysis Summary
===================================

5m Timeframe Analysis:
------------------------------

Best Performing Level: 0.886
- Success Rate: 54.18%
- Expected Return: 144.72%
- Avg Trade Duration: 60.0 candles
- Time to Profit: 32.5 candles
- Risk:Reward: 9.89

Volume Analysis:
- Volume Efficiency: 1.00x
- Most Active Level: 0.236

Risk Metrics:
- Best R:R Level: 0.886
- Volatility: 14.63%

15m Timeframe Analysis:
------------------------------

Best Performing Level: 0.886
- Success Rate: 59.29%
- Expected Return: 142.28%
- Avg Trade Duration: 32.0 candles
- Time to Profit: 19.0 candles
- Risk:Reward: 7.68

Volume Analysis:
- Volume Efficiency: 1.00x
- Most Active Level: 0.236

Risk Metrics:
- Best R:R Level: 0.886
- Volatility: 18.53%

1H Timeframe Analysis:
------------------------------

Best Performing Level: 0.886
- Success Rate: 65.66%
- Expected Return: 155.84%
- Avg Trade Duration: 24.0 candles
- Time to Profit: 15.8 candles
- Risk:Reward: 6.96

Volume Analysis:
- Volume Efficiency: 1.00x
- Most Active Level: 0.382

Risk Metrics:
- Best R:R Level: 0.886
- Volatility: 22.39%

Optimal Trading Parameters:
Timeframe  Best Level  Success Rate  Avg R:R  Hold Time  Vol Efficiency
       5m        0.89          0.54     9.89      60.00            1.00
      15m        0.89          0.59     7.68      32.00            1.00
       1H        0.89          0.66     6.96      24.00            1.00
    """
)

Importing OHLCV data from folder: Data/New_Token_Data/2024_11_06/OHLCV_Data/2024_Nov_06_0224AM

    
Enhanced Fibonacci Analysis Summary

5m Timeframe Analysis:
------------------------------

Best Performing Level: 0.886
- Success Rate: 54.18%
- Expected Return: 144.72%
- Avg Trade Duration: 60.0 candles
- Time to Profit: 32.5 candles
- Risk:Reward: 9.89

Volume Analysis:
- Volume Efficiency: 1.00x
- Most Active Level: 0.236

Risk Metrics:
- Best R:R Level: 0.886
- Volatility: 14.63%

15m Timeframe Analysis:
------------------------------

Best Performing Level: 0.886
- Success Rate: 59.29%
- Expected Return: 142.28%
- Avg Trade Duration: 32.0 candles
- Time to Profit: 19.0 candles
- Risk:Reward: 7.68

Volume Analysis:
- Volume Efficiency: 1.00x
- Most Active Level: 0.236

Risk Metrics:
- Best R:R Level: 0.886
- Volatility: 18.53%

1H Timeframe Analysis:
------------------------------

Best Performing Level: 0.886
- Success Rate: 65.66%
- Expected Return: 155.84%
- Avg Trade Duration:

Fibonacci Strategy 

In [39]:
def plot_volume_analysis(imported_ohlcv_data, initial_timeframe, top_n, volume_multiplier):
    """
    Plot price action with volume analysis, focusing on tokens with significant volume increases.
    Includes 4-period moving average for volume and market cap information.
    
    Parameters:
    - volume_multiplier: Required increase over MA (1.5 = 50% increase)
    """
    
    def calculate_volume_signal(df, ma_period=10, recent_periods=2, volume_multiplier=3):
        """Calculate volume signal and moving average"""
        if len(df) < ma_period + recent_periods:
            return 0, pd.Series([0] * len(df), index=df.index)
        
        # Calculate volume MA
        volume_ma = df['volume'].rolling(window=ma_period).mean()
        
        # Determine which candle to use for signal calculation
        if df['volume'].iloc[-1] == 0:
            # Use previous candle for signal calculation
            current_volume = df['volume'].iloc[-2]
            ma_value = volume_ma.iloc[-3]  # Use MA value before the signal candle
            price_increasing = df['close'].iloc[-2] > df['open'].iloc[-2]
        else:
            # Use latest candle
            current_volume = df['volume'].iloc[-1]
            ma_value = volume_ma.iloc[-2]
            price_increasing = df['close'].iloc[-1] > df['open'].iloc[-1]
        
        # Check for invalid MA value
        if pd.isna(ma_value) or ma_value == 0:
            return 0, volume_ma
        
        volume_increase = (current_volume - ma_value) / ma_value * 100
        volume_significant = current_volume >= (ma_value * volume_multiplier)
        
        return volume_increase if (price_increasing and volume_significant) else 0, volume_ma
    
    # Define enhanced color scheme
    BACKGROUND_COLOR = '#1e1e1e'
    TEXT_COLOR = '#ffffff'
    GRID_COLOR = 'rgba(51, 51, 51, 0.3)'
    TITLE_COLOR = '#00cc00'
    VOLUME_MA_COLOR = '#ffff00'  # Yellow for volume MA
    SEPARATOR_COLOR = '#555555'  # Darker gray for separator
    
    available_timeframes = ['1m', '5m', '15m', '1H']
    token_volume_data = []
    
    # Process market cap data
    base_path = 'Data/New_Token_Data'
    date_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    most_recent_date = max(date_folders)
    date_folder = os.path.join(base_path, most_recent_date)
    
    summary_folder = os.path.join(date_folder, 'Token_Summary')
    datetime_folders = [f for f in os.listdir(summary_folder) if os.path.isdir(os.path.join(summary_folder, f))]
    most_recent_datetime = max(datetime_folders)
    summary_datetime_folder = os.path.join(summary_folder, most_recent_datetime)
    
    mc_files = [f for f in os.listdir(summary_datetime_folder) if f.startswith('new_tokens_mc') and f.endswith('.csv')]
    latest_mc_file = os.path.join(summary_datetime_folder, mc_files[0])
    market_caps = pd.read_csv(latest_mc_file, index_col=0)['Market Cap']
    
    # Calculate volume signals and price changes for all tokens
    for token_address, timeframes in imported_ohlcv_data.items():
        if initial_timeframe not in timeframes:
            continue
            
        df_initial = timeframes[initial_timeframe]
        if df_initial.empty or len(df_initial) < 6:  # Need at least 6 candles
            continue
            
        volume_signal, _ = calculate_volume_signal(df_initial, volume_multiplier=volume_multiplier)
        if pd.isna(volume_signal) or volume_signal == 0:  # Skip if no valid volume signal
            continue
            
        # Calculate price change
        if df_initial['volume'].iloc[-1] == 0:
            # Use previous candle for price change
            price_change = ((df_initial['close'].iloc[-2] - df_initial['open'].iloc[-2]) / 
                           df_initial['open'].iloc[-2] * 100)
        else:
            # Use latest candle
            price_change = ((df_initial['close'].iloc[-1] - df_initial['open'].iloc[-1]) / 
                           df_initial['open'].iloc[-1] * 100)
        
        token_volume_data.append({
            'address': token_address,
            'volume_signal': volume_signal,
            'price_change': price_change
        })
    
    # Sort tokens by price change and take top N
    sorted_tokens = sorted(token_volume_data, 
                          key=lambda x: x['price_change'], 
                          reverse=True)[:top_n]
    
    if not sorted_tokens:
        print("No tokens found meeting the volume and price criteria.")
        return
    
    # Create plots for top tokens
    for token_data in sorted_tokens:
        token_address = token_data['address']
        volume_signal = token_data['volume_signal']
        price_change = token_data['price_change']
        market_cap = Master_Functions.format_number(market_caps.get(token_address, 0))
        dexscreener_link = f"https://dexscreener.com/solana/{token_address}"
        print(f"\nProcessing: {dexscreener_link} (Price Change: {price_change:.2f}% | Volume Signal: {volume_signal:.2f}%)")
        
        timeframes = imported_ohlcv_data[token_address]
        
        # Create subplots with shared x-axis and clear separation
        fig = make_subplots(rows=2, cols=1, 
                           shared_xaxes=True,
                           vertical_spacing=0.08,
                           row_heights=[0.7, 0.3])
        
        for tf in available_timeframes:
            if tf not in timeframes:
                continue
                
            df = timeframes[tf].copy()
            if df.empty:
                continue
            
            # Add candlestick chart without whiskers
            fig.add_trace(
                go.Candlestick(
                    x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'],
                    name="Price",
                    showlegend=False,
                    increasing_line_color='#00ff9d',
                    decreasing_line_color='#ff005b',
                    visible=(tf == initial_timeframe),
                    whiskerwidth=0
                ),
                row=1, col=1
            )
            
            # Calculate volume MA
            _, volume_ma = calculate_volume_signal(df, volume_multiplier=volume_multiplier)
            
            # Calculate colors for volume bars
            colors = ['#00ff9d' if df['close'].iloc[i] >= df['open'].iloc[i] 
                     else '#ff005b' for i in range(len(df))]
            
            # Add volume bars
            fig.add_trace(
                go.Bar(
                    x=df.index,
                    y=df['volume'],
                    name="Volume",
                    marker_color=colors,
                    showlegend=False,
                    visible=(tf == initial_timeframe)
                ),
                row=2, col=1
            )
            
            # Add volume MA line
            if not volume_ma.isna().all():
                fig.add_trace(
                    go.Scatter(
                        x=df.index,
                        y=volume_ma,
                        name="Volume MA (4)",
                        line=dict(color=VOLUME_MA_COLOR, width=2),
                        showlegend=False,
                        visible=(tf == initial_timeframe)
                    ),
                    row=2, col=1
                )
        
        # Update layout with price change included in title
        fig.update_layout(
            height=800,
            width=1200,
            paper_bgcolor=BACKGROUND_COLOR,
            plot_bgcolor=BACKGROUND_COLOR,
            margin=dict(r=50, t=80, l=50, b=50),
            showlegend=False,
            hovermode='x unified',
            title=dict(
                text=f"Price Change: {price_change:.2f}% | Volume Signal: {volume_signal:.2f}% | Market Cap: {market_cap}",
                font=dict(size=16, color=TITLE_COLOR),
                x=0.5,
                y=0.95
            ),
            xaxis_rangeslider_visible=False
        )
        
        # Add separator line between plots
        fig.add_shape(
            type="line",
            x0=0,
            y0=0,
            x1=1,
            y1=0,
            xref="paper",
            yref="paper",
            line=dict(
                color=SEPARATOR_COLOR,
                width=2,
                dash="solid"
            )
        )
        
        # Update axes properties individually
        for i in range(2):
            fig.update_xaxes(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showline=True,
                linecolor=GRID_COLOR,
                row=i+1, col=1
            )
            
            fig.update_yaxes(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showticklabels=True,
                showline=True,
                linecolor=GRID_COLOR,
                zeroline=False,
                fixedrange=True,
                tickfont=dict(color=TEXT_COLOR),
                row=i+1, col=1
            )
        
        fig.show()

# Call the function with 50% volume increase requirement
plot_volume_analysis(imported_ohlcv_data, initial_timeframe='5m', top_n=5, volume_multiplier=5)


Processing: https://dexscreener.com/solana/57VCXi7uAgypyEwDJM3GKp955kBwJrpwbp8xcoVvbjLD (Price Change: 1.02% | Volume Signal: 317.41%)



Processing: https://dexscreener.com/solana/3frx7Ci7bJALYPFNeUGctYxNiarYDuokADbc96VY8XtE (Price Change: 0.72% | Volume Signal: 237.61%)
